In [1]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import PlayerGameLog, leaguedashplayerstats
import pandas as pd
import nba_api.stats.endpoints
import time
import random
import numpy as np

In [2]:
draft_df = nba_api.stats.endpoints.drafthistory.DraftHistory().get_data_frames()[0]

In [3]:
draft_2003 = draft_df[draft_df["SEASON"] == "2003"]
draft_2003

,PERSON_ID,PLAYER_NAME,SEASON,ROUND_NUMBER,ROUND_PICK,OVERALL_PICK,DRAFT_TYPE,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,ORGANIZATION,ORGANIZATION_TYPE,PLAYER_PROFILE_FLAG
1312,2544,LeBron James,2003,1,1,1,Draft,1610612739,Cleveland,Cavaliers,CLE,Saint Vincent-Saint Mary,High School,1
1313,2545,Darko Milicic,2003,1,2,2,Draft,1610612765,Detroit,Pistons,DET,KK Vrsac (Serbia),Other Team/Club,1
1314,2546,Carmelo Anthony,2003,1,3,3,Draft,1610612743,Denver,Nuggets,DEN,Syracuse,College/University,1
1315,2547,Chris Bosh,2003,1,4,4,Draft,1610612761,Toronto,Raptors,TOR,Georgia Tech,College/University,1
1316,2548,Dwyane Wade,2003,1,5,5,Draft,1610612748,Miami,Heat,MIA,Marquette,College/University,1
1317,2549,Chris Kaman,2003,1,6,6,Draft,1610612746,Los Angeles,Clippers,LAC,Central Michigan,College/University,1
1318,2550,Kirk Hinrich,2003,1,7,7,Draft,1610612741,Chicago,Bulls,CHI,Kansas,College/University,1
1319,2551,T.J. Ford,2003,1,8,8,Draft,1610612749,Milwaukee,Bucks,MIL,Texas,College/University,1
1320,2552,Michael Sweetney,2003,1,9,9,Draft,1610612752,New York,Knicks,NYK,Georgetown,College/University,1
1321,2553,Jarvis Hayes,2003,1,10,10,Draft,1610612764,Washington,Wizards,WAS,Georgia,College/University,1


In [4]:
players_name_id = draft_2003[["PLAYER_NAME", "PERSON_ID"]]

In [5]:
players_10_games = []
for player_id, player_name in zip(players_name_id["PERSON_ID"], players_name_id["PLAYER_NAME"]):
	game_log = PlayerGameLog(player_id=player_id, season="2003-04", season_type_all_star="Regular Season")
	df = game_log.get_data_frames()[0]
	df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"])
	df_sorted = df.sort_values("GAME_DATE")
	first_10_games = df_sorted.head(10)
	if not first_10_games.empty:	# Ensuring they have played at least a game
		player_stats = {
			"player_name": player_name,
			"player_id": player_id,
			"fta": first_10_games["FTA"].sum(),
			"ftm": first_10_games["FTM"].sum(),
			"ft_pct": first_10_games["FTM"].sum() / first_10_games["FTA"].sum() if first_10_games["FTA"].sum() > 0 else 0
		}
		players_10_games.append(player_stats)
	time.sleep(random.uniform(0.5, 1.5))  # To avoid hitting rate limits
df_players_10_games = pd.DataFrame(players_10_games)

In [6]:
# removing players who didnt shoot free throws in their first 10 games
# since they dont provide meaningful data for free throw percentage
df_players_10_games = df_players_10_games[df_players_10_games["fta"] > 0]
df_players_10_games

,player_name,player_id,fta,ftm,ft_pct
0,LeBron James,2544,38,23,0.605263
1,Darko Milicic,2545,3,1,0.333333
2,Carmelo Anthony,2546,60,43,0.716667
3,Chris Bosh,2547,27,23,0.851852
4,Dwyane Wade,2548,36,24,0.666667
5,Chris Kaman,2549,17,12,0.705882
6,Kirk Hinrich,2550,22,15,0.681818
7,T.J. Ford,2551,17,15,0.882353
8,Michael Sweetney,2552,6,5,0.833333
9,Jarvis Hayes,2553,11,9,0.818182


Because each player is a binomial and their free throw variance is different we need to apply shrinkage differently to each player.

Reason for applying different shrinkages is beacause if:
Player A:
- has shot 200FT

Player B:
- has shot 20FT

it is not accurate to apply the same shrinkage to both of them

shrinkage equation

$c = 1 - \frac{(k - 3)\sigma^2}{\sum (y - \bar{y})^2}$

$z = \bar{y} + c (y - \bar{y})$

where:
- z is the JS adjusted average
- $\bar{y}$ is the "grand" average


In [ ]:
# applying james stein
# lets try with changing shrinkage c
grand_average = df_players_10_games["ft_pct"].mean()
k = len(df_players_10_games)

def js_avg(df):
    # free throws are a binomial process
	variance = df["ft_pct"] * (1-df["ft_pct"]) / df["fta"]

	shrinkage_c = 1 - ((k-3)*variance) / sum((df["ft_pct"] - grand_average)**2)
	# apply a sigmoid function here to limit shrinkage c between 0 and 1
	shrinkage_c = 1 / (1 + np.exp(shrinkage_c))
	df["variance"] = variance
	df["shrinkage_c"] = shrinkage_c
	df["ft_pct_js"] = grand_average + shrinkage_c * (df["ft_pct"] - grand_average)

	return df

df_players_10_games_js = js_avg(df_players_10_games)

In [8]:
stats = leaguedashplayerstats.LeagueDashPlayerStats(
    season="2003-04",
    per_mode_detailed="Totals"
)

stats_df = stats.get_data_frames()[0]
rookie_2003_stats = stats_df[stats_df["PLAYER_ID"].isin(df_players_10_games["player_id"])]

In [9]:
rookie_2003_stats

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,TEAM_COUNT
39,2564,Boris Diaw,Boris,1610612737,ATL,22.0,76,28,48,0.368,...,184,361,79,232,429,175,178,19,180,1
43,2599,Brandon Hunter,Brandon,1610612738,BOS,23.0,36,14,22,0.389,...,192,96,79,332,187,335,178,19,335,1
48,2567,Brian Cook,Brian,1610612747,LAL,23.0,35,23,12,0.657,...,64,139,79,318,191,315,178,19,320,1
61,2546,Carmelo Anthony,Carmelo,1610612743,DEN,20.0,82,43,39,0.524,...,441,404,21,6,96,20,53,19,16,1
70,2547,Chris Bosh,Chris,1610612761,TOR,20.0,75,29,46,0.387,...,401,392,79,93,398,82,46,19,91,1
73,2549,Chris Kaman,Chris,1610612746,LAC,22.0,82,28,54,0.341,...,392,386,79,174,420,152,86,19,151,1
87,2563,Dahntay Jones,Dahntay,1610612763,MEM,23.0,20,13,7,0.650,...,75,70,79,381,271,377,235,19,380,1
101,2545,Darko Milicic,Darko,1610612765,DET,19.0,34,29,5,0.853,...,133,84,79,371,276,362,235,19,366,1
106,2561,David West,David,1610612740,NOH,23.0,71,35,36,0.493,...,256,229,79,253,347,228,146,19,232,1
127,2548,Dwyane Wade,Dwyane,1610612748,MIA,22.0,61,34,27,0.557,...,412,263,79,76,75,96,178,19,96,1


In [10]:
# calculating loss
ground_truth = rookie_2003_stats[["PLAYER_ID", "FT_PCT"]]
ground_truth = ground_truth.rename(columns={"PLAYER_ID": "player_id", "FT_PCT": "ft_pct_truth"})

In [11]:
df = df_players_10_games_js.merge(
    ground_truth[["player_id", "ft_pct_truth"]],
    on="player_id",
    how="inner"
)

In [12]:
loss = ((df["ft_pct_truth"] - df["ft_pct"]) ** 2).mean()
loss


np.float64(0.01833137903477564)

In [13]:
loss_js = ((df["ft_pct_truth"] - df["ft_pct_js"]) ** 2).mean()
loss_js

np.float64(0.007360226614887888)

In [ ]:
df = df.drop(columns=["variance", "player_id"])

In [ ]:
df.to_csv("2003_players_ft.csv", index=False)

In [17]:
df["ft_pct_truth"] - df["ft_pct"]

0     0.148737
1     0.249667
2     0.060333
3    -0.150852
4     0.080333
5    -0.008882
6     0.122182
7    -0.066353
8    -0.109333
9    -0.032182
10    0.104765
11    0.019158
12    0.023000
13    0.140000
14    0.000000
15    0.017143
16    0.025500
17   -0.226000
18    0.055000
19    0.102000
20   -0.033667
21    0.000000
22   -0.073529
23    0.000333
24    0.270000
25    0.274429
26    0.041727
27   -0.167000
28   -0.095000
29   -0.185000
30   -0.179000
31    0.028000
32   -0.106000
33    0.297667
34    0.186000
35    0.000000
36   -0.208000
37    0.108667
dtype: float64